In [7]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
import featuretools as ft
import numpy as np

## frequency of monthly wildfire by states ##

In [8]:
df = pd.read_csv('./DATA/etl_filtered_v1.csv')
df['month'] = pd.DatetimeIndex(df['datetime']).month
df

,Unnamed: 0,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,CONT_DATE,DISCOVERY_DATE,CONT_DOY,DISCOVERY_TIME,CONT_TIME,...,STATE,datetime,population_density,ELEVATION,AWND,PRCP,TMAX,TMIN,distance,month
0,0,2011,47,Debris Burning,5.0,2455608.5,2455608.5,47.0,1300.0,1530.0,...,AL,2011-02-16,7.572142,100.6,8.062714,0.0,71.000000,32.000000,0.141421,2
1,1,2011,47,Debris Burning,5.0,2455610.5,2455608.5,49.0,820.0,1500.0,...,AL,2011-02-16,31.442815,201.5,8.048481,0.0,76.942800,49.340830,0.100000,2
2,2,2011,48,Arson,7.0,2455609.5,2455609.5,48.0,2030.0,2300.0,...,AL,2011-02-17,7.572142,100.6,8.064342,0.0,70.000000,33.000000,0.200000,2
3,3,2011,50,Arson,7.0,2455617.5,2455611.5,56.0,1800.0,1500.0,...,AL,2011-02-19,49.942890,182.9,8.037152,0.0,66.000000,49.000000,0.282843,2
4,4,2011,51,Debris Burning,5.0,2455625.5,2455612.5,64.0,1500.0,1900.0,...,AL,2011-02-20,49.942890,189.0,8.042611,0.0,76.657125,49.084371,0.100000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367739,367739,2015,75,Miscellaneous,9.0,NaN,2457097.5,NaN,NaN,NaN,...,VA,2015-03-16,33.568885,283.5,7.366097,0.0,61.000000,39.000000,0.100000,3
367740,367740,2015,95,Miscellaneous,9.0,NaN,2457117.5,NaN,NaN,NaN,...,VA,2015-04-05,33.568885,283.5,7.358285,0.0,57.000000,39.000000,0.100000,4
367741,367741,2015,37,Miscellaneous,9.0,NaN,2457059.5,NaN,NaN,NaN,...,VA,2015-02-06,38.354455,247.8,7.406834,0.0,70.218508,43.916184,0.100000,2
367742,367742,2015,89,Miscellaneous,9.0,NaN,2457111.5,NaN,NaN,NaN,...,VA,2015-03-30,38.354455,208.2,7.384267,0.0,69.862021,43.603630,0.141421,3


In [ ]:
states = list(set(df.STATE.values))
for i in states:
    temp_df = df[df['STATE']==i]
    temp_df = temp_df.groupby(['FIRE_YEAR', 'month']).agg(frequency=('FIRE_YEAR', 'count'))
    dir = './DATA/monthly_with_year/'
    if not os.path.exists(dir):
        os.mkdir(dir)
    path_name = dir + i + '.csv'
    temp_df.to_csv(path_name)
for i in states:
    temp_df = df[df['STATE']==i]
    temp_df = temp_df.groupby(['month']).agg(frequency=('FIRE_YEAR', 'count'))
    dir = './DATA/monthly_without_year/'
    if not os.path.exists(dir):
        os.mkdir(dir)
    path_name = dir + i + '.csv'
    temp_df.to_csv(path_name)

In [13]:
year = list(range(2011,2016))
for i in year:
    temp_df = df[df['FIRE_YEAR']==i]
    temp_df = temp_df.groupby(['STATE', 'month']).size().reset_index(name='counts')
    dir = './DATA/monthly_by_year/'
    if not os.path.exists(dir):
        os.mkdir(dir)
    path_name = dir + str(i) + '.csv'
    temp_df.to_csv(path_name)

In [ ]:
# encoding state
enc = OneHotEncoder(handle_unknown='ignore')
X = df['STATE'].to_numpy().reshape(-1, 1)
enc.fit(X)
ec_df = pd.DataFrame(enc.transform(X).toarray())
ec_df = df.join(ec_df)
ec_df

In [ ]:
df.columns

In [ ]:
mean_pd = df[df['population_density']>=0].groupby('STATE')['population_density'].mean().reset_index()
# print(mean_pd)
ml_df = df[['month', 'STATE', 'TMAX', 'TMIN', 'AWND']]
ml_df = ml_df.groupby(['month', 'STATE']).agg({'month' : ['count'], 'TMAX' : 'mean', 'TMIN' : 'mean', 'AWND' : 'mean'}).reset_index()
# ml_df.columns = ml_df.columns.get_level_values(0)
ml_df.columns = ['_'.join(col).strip() for col in ml_df.columns.values]
# print(ml_df)
X = ml_df.merge(mean_pd, how='inner', left_on='STATE_', right_on='STATE')
encoding = pd.get_dummies(X.STATE)
X = X.join(encoding)
X

In [ ]:
X.columns

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn import linear_model
y = X['month_count']
X = X.drop(['STATE_', 'month_count', 'STATE'], axis=1)

In [ ]:
X

In [ ]:
model = BayesianRidge()
model.fit(X, y)
model.score(X, y)

In [ ]:
df.columns

In [ ]:
#predict
temp = df[['FIRE_YEAR', 'month', 'STATE', 'AWND', 'TMAX', 'TMIN']]
AWND_predict = temp.groupby(['FIRE_YEAR', 'month', 'STATE']).agg({'AWND' : 'mean'}).reset_index()
encoding = pd.get_dummies(AWND_predict.STATE)
AWND_predict = AWND_predict.join(encoding)
y = AWND_predict['AWND']
x = AWND_predict.drop(['STATE', 'AWND'], axis=1)
AWND_model = LinearRegression().fit(x, y)

In [ ]:
TMAX_predict = temp.groupby(['FIRE_YEAR', 'month', 'STATE']).agg({'TMAX' : 'mean'}).reset_index()
encoding = pd.get_dummies(TMAX_predict.STATE)
TMAX_predict = TMAX_predict.join(encoding)
y = TMAX_predict['TMAX']
x = TMAX_predict.drop(['STATE', 'TMAX'], axis=1)
TMAX_model = LinearRegression().fit(x, y)

In [ ]:
TMIN_predict = temp.groupby(['FIRE_YEAR', 'month', 'STATE']).agg({'TMIN' : 'mean'}).reset_index()
encoding = pd.get_dummies(TMIN_predict.STATE)
TMIN_predict = TMIN_predict.join(encoding)
y = TMIN_predict['TMIN']
x = TMIN_predict.drop(['STATE', 'TMIN'], axis=1)
TMIN_model = LinearRegression().fit(x, y)

In [ ]:
x

In [ ]:
temp

In [ ]:
states = set(df['STATE'].to_list())
states = list(states)
states.sort()
year = [2022]
months = list(range(1,13))
year_df = pd.DataFrame(year, columns=['FIRE_YEAR'])
month_df = pd.DataFrame(months, columns=['month_'])
State_df = pd.DataFrame(states, columns=['STATE'])
raw = year_df.merge(month_df, how='cross')
raw = raw.merge(State_df, how='cross')

encoding = pd.get_dummies(raw.STATE)
input_df = raw.join(encoding)
input_df = input_df.drop(['STATE'], axis=1)
input_df

In [ ]:
raw

In [ ]:
AWND_2022 = AWND_model.predict(input_df)
TMIN_2022 = TMIN_model.predict(input_df)
TMAX_2022 = TMAX_model.predict(input_df)

In [ ]:
# TMAX_mean	TMIN_mean	AWND_mean
raw['TMAX_mean'] = TMAX_2022
raw['TMIN_mean'] = TMIN_2022
raw['AWND_mean'] = AWND_2022
mean_pd = df[df['population_density']>=0].groupby('STATE')['population_density'].mean().reset_index()
raw = raw.merge(mean_pd, how='inner', on='STATE')
raw.join(pd.get_dummies(raw.STATE))
input_2022 = raw.join(pd.get_dummies(raw.STATE))
input_2022
input_2022 = input_2022.drop(['STATE', 'FIRE_YEAR'], axis=1)
input_2022

In [ ]:
frequency_2022 = model.predict(input_2022)
raw['frequency'] = frequency_2022
raw['frequency'] = raw['frequency'].apply(lambda x: int(x) if x>=0 else 0)
raw[['FIRE_YEAR', 'month_', 'STATE', 'frequency']].to_csv('frequency_2022.csv')